# Introduction

There are many metrics used to evaluate the worth of a company (P/E ratios, EPS, n-day moving averages, etc), is there a way to apply some statistical methods using a company's financial data to predict the value of a company?

The first task is finding the data needed to answer the question. Using SimFin, I had an easy way of bulk downloading all the financial data I needed onto various CSV files. With the free version of their API, the data excludes the most recent year's information. 

Next, I focused on importing the data and cleaning it so that it can be used in the model. Since SimFin already standarized the layout in the CSV files, little is needed in this area. However, there are various data missing for either rows or columns, so they will need to be addressed before running the model. In Model A, the quickest approach was to fill the value of 0 for anything that was missing data. 

Another issue with the data is that the 3 financial statements (cash flow, balance sheet, income statement) are on 3 separate files and the companies are broken up into 3 categories (banks, insurances, and all others). Since there is the assumption that the 3 financial statement's data are needed for the model, these 3 files will need to be merged into 1 DataFrame. The same cannot be said about the 3 categories as they each use different columns and capture different information. This will not be an issue as Model A will only be using 2 points of reference, the company's historical data by itself and the company's sharing the same industry as the target company. Since the industry of a company will not cross over to another category, there will be 3 separate DataFrames for the 3 categories.

# Downloading Data

In [1]:
import simfin as sf
from IPython.display import clear_output

# Set your API-key for downloading data. This key gets the free data.
sf.set_api_key('free') 
# Set the local directory where data-files are stored.
# The directory will be created if it does not already exist.
sf.set_data_dir('./stocks/')

sf.load_income(variant='quarterly', market='us')
sf.load_income_banks(variant='quarterly', market='us')
sf.load_income_insurance(variant='quarterly', market='us')

sf.load_balance(variant='quarterly', market='us')
sf.load_balance_banks(variant='quarterly', market='us')
sf.load_balance_insurance(variant='quarterly', market='us')

sf.load_cashflow(variant='quarterly', market='us')
sf.load_cashflow_banks(variant='quarterly', market='us')
sf.load_cashflow_insurance(variant='quarterly', market='us')

sf.load_shareprices(variant='daily', market='us')
sf.load_companies(market='us')
sf.load_industries()

clear_output()
print('Data download complete')

Data download complete


# Importing Data into DataFrames and Data Cleaning

## Importing Bank Specific Financial Statements

In [2]:
from functools import reduce
import pandas as pd
import numpy as np

#Creates a DataFrame of banks' balance sheets
bank_bal_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', 'Report Date', 'Shares (Basic)', \
                'Cash, Cash Equivalents & Short Term Investments', 'Interbank Assets', \
                'Short & Long Term Investments', 'Net Loans', 'Net Fixed Assets', \
                'Total Deposits', 'Short Term Debt', 'Long Term Debt', \
                'Preferred Equity', 'Share Capital & Additional Paid-In Capital', 'Treasury Stock', \
                'Retained Earnings']
bank_bal = pd.read_csv('./stocks/us-balance-banks-quarterly.csv', sep=';', \
                       usecols=bank_bal_col, parse_dates=['Report Date'])

#Since the entire DF is bank companies, a column called IndustryId was created with a single value
#This is to ensure the model code doesn't need to be modified between the 3 DF
bank_bal['IndustryId'] = 999

#Brings in share prices for market cap calculations
price_col = ['Ticker', 'Date', 'Close']
price = pd.read_csv('./stocks/us-shareprices-daily.csv', usecols=price_col, sep=';', parse_dates=['Date'])
#Uses the method merge_asof since some financial statement reports come out when exchanges are closed
#Parameters will do its best to match the Report Date to Share Date, if it cannot find, it will look backwards
bank_bal = pd.merge_asof(left=bank_bal.sort_values('Report Date'), right=price.sort_values('Date'), \
                         left_on='Report Date', right_on='Date', \
                         left_by='Ticker', right_by='Ticker')
bank_bal.sort_values(by=['Ticker', 'Report Date'], inplace=True)
bank_bal['Market Cap'] = bank_bal['Shares (Basic)'] * bank_bal['Close']
bank_bal.dropna(subset=['Market Cap', 'Shares (Basic)'], inplace=True)


bank_bal.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Creates a DataFrame of banks' income statements
bank_inc_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', 'Revenue', 'Provision for Loan Losses', \
                'Net Revenue after Provisions', 'Operating Income (Loss)', \
                'Pretax Income (Loss)', 'Income Tax (Expense) Benefit, Net', \
                'Income (Loss) from Continuing Operations', \
                'Net Income']
bank_inc = pd.read_csv('./stocks/us-income-banks-quarterly.csv', sep=';', usecols=bank_inc_col)
bank_inc.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Creates a DataFrame of banks' cash flow statements
bank_cash_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', 'Depreciation & Amortization', \
                 'Non-Cash Items', 'Change in Working Capital', 'Net Cash from Operating Activities', \
                 'Change in Fixed Assets & Intangibles', 'Net Change in Loans & Interbank', \
                 'Net Cash from Investing Activities', 'Dividends Paid', \
                 'Cash from (Repayment of) Debt', 'Cash from (Repurchase of) Equity', \
                 'Net Cash from Financing Activities', 'Net Change in Cash']
bank_cash = pd.read_csv('./stocks/us-cashflow-banks-quarterly.csv', sep=';', usecols=bank_cash_col)
bank_cash.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Merges the 3 financials statements into 1 DataFrame
bank_df = [bank_cash, bank_inc, bank_bal]
bank = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True,how='inner'), bank_df).fillna(0)

print('Bank DataFrame creation complete.')

Bank DataFrame creation complete.


## Importing Insurance Specific Financial Statements

In [3]:
from functools import reduce
import pandas as pd
import numpy as np

#Creates a DataFrame of insurances' balance sheets
ins_bal_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', 'Report Date', 'Shares (Basic)', 'Total Investments', \
               'Cash, Cash Equivalents & Short Term Investments', 'Accounts & Notes Receivable', \
               'Total Assets', 'Insurance Reserves', 'Long Term Debt', 'Total Liabilities', \
               'Share Capital & Additional Paid-In Capital', 'Retained Earnings', 'Total Equity', \
               'Total Liabilities & Equity']
ins_bal = pd.read_csv('./stocks/us-balance-insurance-quarterly.csv', sep=';', \
                       usecols=ins_bal_col, parse_dates=['Report Date'])

#Since the entire DF is insurance companies, a column called IndustryId was created with a single value
#This is to ensure the model code doesn't need to be modified between the 3 DF
ins_bal['IndustryId'] = 888

#Brings in share prices for market cap calculations
price_col = ['Ticker', 'Date', 'Close']
price = pd.read_csv('./stocks/us-shareprices-daily.csv', usecols=price_col, sep=';', parse_dates=['Date'])
#Uses the method merge_asof since some financial statement reports come out when exchanges are closed
#Parameters will do its best to match the Report Date to Share Date, if it cannot find, it will look backwards
ins_bal = pd.merge_asof(left=ins_bal.sort_values('Report Date'), right=price.sort_values('Date'), \
                        left_on='Report Date', right_on='Date', \
                        left_by='Ticker', right_by='Ticker')
ins_bal.sort_values(by=['Ticker', 'Report Date'], inplace=True)
ins_bal['Market Cap'] = ins_bal['Shares (Basic)'] * ins_bal['Close']
ins_bal.dropna(subset=['Market Cap', 'Shares (Basic)'], inplace=True)

ins_bal.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Creates a DataFrame of insurances' income statements
ins_inc_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', 'Revenue', 'Total Claims & Losses', \
               'Operating Income (Loss)', 'Pretax Income (Loss)', 'Income Tax (Expense) Benefit, Net', \
               'Income (Loss) from Continuing Operations']
ins_inc = pd.read_csv('./stocks/us-income-insurance-quarterly.csv', sep=';', usecols=ins_inc_col)
ins_inc.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Creates a DataFrame of insurances' cash flow statements
ins_cash_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', 'Net Income/Starting Line', \
                'Depreciation & Amortization', 'Non-Cash Items', 'Net Change in Investments', \
                'Net Cash from Investing Activities', 'Dividends Paid', 'Cash from (Repayment of) Debt', \
                'Cash from (Repurchase of) Equity', 'Net Cash from Financing Activities', 'Net Change in Cash']
ins_cash = pd.read_csv('./stocks/us-cashflow-insurance-quarterly.csv', sep=';', usecols=ins_cash_col)
ins_cash.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Merges the 3 financials statements into 1 DataFrame
ins_df = [ins_cash, ins_inc, ins_bal]
ins = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True,how='inner'), ins_df).fillna(0)

print('Insurance DataFrame creation complete')

Insurance DataFrame creation complete


## Importing Companies' Financial Statements

In [1]:
from functools import reduce
import pandas as pd
import numpy as np


#Creates a DataFrame of companies balance sheets
comp_bal_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', 'Report Date', 'Shares (Basic)', \
                'Cash, Cash Equivalents & Short Term Investments', 'Accounts & Notes Receivable', \
                'Inventories', 'Total Current Assets', 'Property, Plant & Equipment, Net', \
                'Other Long Term Assets', 'Total Noncurrent Assets', 'Total Assets', 'Payables & Accruals', \
                'Short Term Debt', 'Total Current Liabilities', 'Long Term Debt', 'Total Noncurrent Liabilities', \
                'Total Liabilities', 'Share Capital & Additional Paid-In Capital', 'Retained Earnings', \
                'Total Equity', 'Total Liabilities & Equity']
comp_bal = pd.read_csv('./stocks/us-balance-quarterly.csv', sep=';', \
                       usecols=comp_bal_col, parse_dates=['Report Date'])

#Add the industry type to each company's ticker
industry = pd.read_csv('./stocks/industries.csv', sep=';')
comp_ind_col = ['Ticker', 'IndustryId']
comp_ind = pd.read_csv('./stocks/us-companies.csv', sep=';', usecols=comp_ind_col)
comp_bal = comp_bal.merge(right=comp_ind, how='left', on='Ticker')

#Brings in share prices for market cap calculations
price_col = ['Ticker', 'Date', 'Close']
price = pd.read_csv('./stocks/us-shareprices-daily.csv', usecols=price_col, sep=';', parse_dates=['Date'])
#Uses the method merge_asof since some financial statement reports come out when exchanges are closed
#Parameters will do its best to match the Report Date to Share Date, if it cannot find, it will look backwards
comp_bal = pd.merge_asof(left=comp_bal.sort_values('Report Date'), right=price.sort_values('Date'), \
                         left_on='Report Date', right_on='Date', \
                         left_by='Ticker', right_by='Ticker')
comp_bal.sort_values(by=['Ticker', 'Report Date'], inplace=True)
comp_bal['Market Cap'] = comp_bal['Shares (Basic)'] * comp_bal['Close']
comp_bal.dropna(subset=['Market Cap', 'Shares (Basic)'], inplace=True)

comp_bal.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Creates a DataFrame of companies income statements
comp_inc_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', 'Revenue', \
                'Cost of Revenue', 'Gross Profit', 'Operating Expenses', 'Selling, General & Administrative', \
                'Operating Income (Loss)', 'Non-Operating Income (Loss)', 'Interest Expense, Net', \
                'Pretax Income (Loss), Adj.', 'Pretax Income (Loss)', 'Income Tax (Expense) Benefit, Net', \
                'Income (Loss) from Continuing Operations', 'Net Income']
comp_inc = pd.read_csv('./stocks/us-income-quarterly.csv', sep=';', usecols=comp_inc_col)
comp_inc.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Creates a DataFrame of companies cash flow statements
comp_cash_col = ['Ticker', 'Fiscal Year', 'Fiscal Period', \
                 'Net Income/Starting Line', 'Depreciation & Amortization', 'Non-Cash Items', \
                 'Change in Working Capital', 'Net Cash from Operating Activities', \
                 'Change in Fixed Assets & Intangibles', 'Net Cash from Investing Activities', \
                 'Dividends Paid', 'Cash from (Repayment of) Debt', 'Cash from (Repurchase of) Equity', \
                 'Net Cash from Financing Activities', 'Net Change in Cash']
comp_cash = pd.read_csv('./stocks/us-cashflow-quarterly.csv', sep=';', usecols=comp_cash_col)
comp_cash.set_index(['Ticker', 'Fiscal Year', 'Fiscal Period'], inplace=True)

#Merges all 3 financial statements into 1 DataFrame
comp_df = [comp_cash, comp_inc, comp_bal]
comp = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True,how='inner'), comp_df)

print('Company DataFrame creation complete.')

Company DataFrame creation complete.


# Model A: Linear Regression Model Assuming Missing Data as Zeros

In [5]:
from sklearn import linear_model

def predictor(ticker):
    
    df = pd.DataFrame()
    
    #Lookup which DataFrame a stock ticker belongs to
    try:
        if len(bank.loc[ticker]) > 0: df = bank
    except:
        try:
            if len(ins.loc[ticker]) > 0: df = ins
        except:
            try:
                if len(comp.loc[ticker]) > 0: df = comp
            except:
                pass
    
    #If a ticker cannot be found, then it will return a row of 0's 
    if len(df) == 0:
        print('Ticker symbol not found in dataset')
        return pd.DataFrame(np.array([[ticker, 0, 0, 0, 0]]), \
                            columns = ['Ticker', 'Date', 'Self', 'Industry', 'Actual'])
    
    #If the industry value is missing, then it will return a row of 0's
    industry = df.loc[ticker, 'IndustryId'].unique().item()
    if np.isnan(industry):
        return pd.DataFrame(np.array([[ticker, 0, 0, 0, 0]]), \
                            columns = ['Ticker', 'Date', 'Self', 'Industry', 'Actual'])
    
    
    #Main DF for modelling based on the industry
    Xi = df.loc[df['IndustryId'] == industry].fillna(0)
    
    #Main DF for modelling based on the company only
    Xs = Xi.loc[ticker]
    
    #Pulls out the row from the dataset to test
    recent = Xs['Report Date'].max()
    X_test = Xs[Xs['Report Date'] == recent]
    y_test = X_test['Market Cap']
    y_actual = round(y_test.to_numpy().item())
    
    #Creates 2 DF for the self model excluding the row of financial data for the hypothesis test
    Xs_train = Xs[Xs['Report Date'] != recent]
    ys_train = Xs_train[['Market Cap']]
    
    #Creates 2 DF for the industry model excluding the row of financial data for the hypothesis test
    Xi_train = Xi[(Xi['Report Date'] != recent) & (Xi.index.get_level_values('Ticker') != ticker)]
    yi_train = Xi_train[['Market Cap']]
    
    #Removes non-financial statement columns from the regressor DF
    rm_col = ['Shares (Basic)', 'Date', 'Close', 'Report Date', 'Market Cap', 'IndustryId']
    Xs_train.drop(rm_col, axis=1, inplace=True)
    Xi_train.drop(rm_col, axis=1, inplace=True)
    X_test.drop(rm_col, axis=1, inplace=True)
    Xi_train.to_csv('1.csv')
    
    if len(Xi_train) == 0 or len(Xs_train) == 0:
        return pd.DataFrame(np.array([[ticker, 0, 0, 0, 0]]), \
                            columns = ['Ticker', 'Date', 'Self', 'Industry', 'Actual'])
    
    #Linear regression model, prediction, and score based only on the company's historical statements
    lm = linear_model.LinearRegression()
    lm_s = lm.fit(Xs_train, ys_train)
    prediction_s = round(lm_s.predict(X_test).item())
    #score_s = lm_s.score(X=Xs_train, y=ys_train)
    #print(score_s)
    
    #Linear regression model, prediction, and score based only on the industry's historical statements
    lm_i = lm.fit(Xi_train, yi_train)
    prediction_i = round(lm_i.predict(X_test).item())
    #score_i = lm_i.score(X=Xi_train, y=yi_train)
    #print(score_i)
    
    return pd.DataFrame(np.array([[ticker, recent, prediction_s, prediction_i, y_actual]]), \
                        columns = ['Ticker', 'Date', 'Self', 'Industry', 'Actual'])

# Run Model A Using All Known Companies

In [39]:
#This code can take several minutes to fully run

#All warnings are suppressed as I am unable to fix the issue of SettingWithCopyWarning
warnings.filterwarnings('ignore')
my_df = pd.DataFrame()
tickers = list(comp.index.get_level_values('Ticker').unique())
for name in tickers:
    my_df = pd.concat([my_df, predictor(name)], axis=0)
warnings.filterwarnings('default')
my_df.set_index(keys='Ticker', inplace=True)
print('Prediction DataFrame created')

Prediction DataFrame created


In [61]:
my_df.sample(frac = 1/200, random_state=2)

,Date,Self,Industry,Actual
Ticker,,,,
BLK,2019-05-31 00:00:00,-176783833452689088,-46496412454,65769864209
NOVT,2019-06-30 00:00:00,2896808089,806050196,3300311400
NE,2019-06-30 00:00:00,159184508,3810897141,465917980
VRNS,2019-06-30 00:00:00,2663230179,1594848046,1875817037
GRMM,0,0,0,0
COOL,2018-12-31 00:00:00,3295811965,-408373778,3784110745
HNRG,2019-06-30 00:00:00,169303791,284241535,170279350
JWN,2019-05-31 00:00:00,5076317600,35862355427,4851500000
PRGO,2013-09-30 00:00:00,15832611963,6882176782,11622396000


In [7]:
predictor("CMI")

C:\Users\Allen\anaconda3\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Ticker,Date,Self,Industry,Actual
0,CMI,2019-06-30,21435249573,35008659681,26968916000


# Testing Code Below

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model

pca = PCA(0.80)
PC = pca.fit_transform(bank_x)
x = pd.DataFrame(PC)
y = bank_y

lm = linear_model.LinearRegression()
model = lm.fit(x, y)
predictions = lm.predict(x.iloc[:1])
print(predictions[0:5])
print(bank_y[0:5])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(bank_x, bank_y, test_size=0.2, random_state=42)
pca = PCA(0.80)
PC = pca.fit_transform(X_train)
x = pd.DataFrame(PC)
y = y_train

PC_X_test = pca.fit_transform(X_test)
pca_X_test =  pd.DataFrame(PC_X_test)

lm = linear_model.LinearRegression()
model = lm.fit(x, y)
print(lm.score(pca_X_test,y_test))

predicts = lm.predict(pca_X_test)
print(predicts[0:5])
print(y_test[0:5])

In [ ]:
jpm_x = bank.loc['JPM'].copy()
jpm_x.drop(['Shares (Basic)', 'Date', 'Close', 'Report Date'], axis=1, inplace=True)
jpm_x = StandardScaler().fit_transform(jpm_x)
#jpm_y = jpm_x[['Market Cap']].copy()
#jpm_x.drop(['Market Cap'], axis=1, inplace=True)
#print(jpm_y.head())
print(jpm_x.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model

pca = PCA(0.9)
PC = pca.fit_transform(jpm_x)
x = pd.DataFrame(PC)
y = jpm_y

lm = linear_model.LinearRegression()
model = lm.fit(x, y)

print(lm.score(x,y))

predictions = lm.predict(x)
print(predictions[0:5])
print(jpm_y[0:5])

In [9]:
comp

Net Income/Starting Line  \
Ticker Fiscal Year Fiscal Period                             
 AVLR  2018        Q2                        -1.776600e+07   
                   Q3                        -2.410300e+07   
                   Q4                        -1.843200e+07   
       2019        Q1                        -9.243000e+06   
                   Q2                        -1.317300e+07   
...                                                    ...   
low    2018        Q2                         1.521000e+09   
                   Q3                         6.290000e+08   
                   Q4                        -8.240000e+08   
       2019        Q1                         1.046000e+09   
                   Q2                         1.676000e+09   

                                  Depreciation & Amortization  Non-Cash Items  \
Ticker Fiscal Year Fiscal Period                                                
 AVLR  2018        Q2                               3002000.0    3.095000e+06   
                   Q3                               3398000.0    1.335600e+07   
                   Q4                               3412000.0    4.440000e+06   
       2019        Q1                               3681000.0    6.746000e+06   
                   Q2                               3987000.0    1.004500e+07   
...                                                       ...             ...   
low    2018        Q2                             364000000.0    2.420000e+08   
                   Q3                             455000000.0    9.500000e+07   
                   Q4                             401000000.0    1.168000e+09   
       2019        Q1                             337000000.0    4.600000e+07   
                   Q2                             347000000.0    1.620000e+08   

                                  Change in Working Capital  \
Ticker Fiscal Year Fiscal Period                              
 AVLR  2018        Q2                          1.376800e+07   
                   Q3                          8.527000e+06   
                   Q4                          1.761600e+07   
       2019        Q1                         -1.160500e+07   
                   Q2                          9.141000e+06   
...                                                     ...   
low    2018        Q2                          2.310000e+08   
                   Q3                         -1.680000e+08   
                   Q4                         -1.350000e+09   
       2019        Q1                          7.080000e+08   
                   Q2                         -7.390000e+08   

                                  Net Cash from Operating Activities  \
Ticker Fiscal Year Fiscal Period                                       
 AVLR  2018        Q2                                   2.099000e+06   
                   Q3                                   1.178000e+06   
                   Q4                                   7.036000e+06   
       2019        Q1                                  -1.042100e+07   
                   Q2                                   1.000000e+07   
...                                                              ...   
low    2018        Q2                                   2.358000e+09   
                   Q3                                   1.011000e+09   
                   Q4                                  -6.050000e+08   
       2019        Q1                                   2.137000e+09   
                   Q2                                   1.446000e+09   

                                  Change in Fixed Assets & Intangibles  \
Ticker Fiscal Year Fiscal Period                                         
 AVLR  2018        Q2                                       -4544000.0   
                   Q3                                       -4745000.0   
                   Q4                                       -2569000.0   
       2019        Q1                         